In [31]:
import pandas as pd
import requests
import time
from datetime import datetime

def log(msg):
    print(f"[{datetime.now()}] {msg}")

# ----------- Paths & File Names -----------
MASTER_PATH       = "steam_games_with_owners_parsed.csv"
CLEANED_APPIDS_PATH = "steam_appids_clean.csv"  # contains columns “appid” and “name”
LOG_PATH          = "steam_games_merge_log.txt"
BATCH_SIZE        = 1700

# ----------- Step 1: Sample App IDs -----------
log("STEP 1: Sampling random App IDs from cleaned list...")
app_df = pd.read_csv(CLEANED_APPIDS_PATH)
log(f"  → Loaded {len(app_df)} total App IDs.")
sample_df = app_df.sample(n=BATCH_SIZE, random_state=int(time.time())).reset_index(drop=True)
log(f"  → Sampled {len(sample_df)} App IDs: {list(sample_df['appid'])[:10]}...")

# ----------- Step 2: Fetch Steam Storefront Data -----------
log("STEP 2: Fetching Steam storefront data for sampled App IDs...")
app_ids = sample_df["appid"].astype(int).tolist()
steam_rows = []
MAX_RETRIES = 2
INITIAL_WAIT = 2  # seconds

for idx, app_id in enumerate(app_ids, start=1):
    log(f"  [{idx}/{len(app_ids)}] Fetching Steam info for App ID {app_id}...")
    retry = 0
    success = False
    while retry < MAX_RETRIES and not success:
        try:
            url = f"https://store.steampowered.com/api/appdetails?appids={app_id}&cc=us&l=en"
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            if str(app_id) in data and data[str(app_id)].get("success"):
                info = data[str(app_id)]["data"]
                if info.get("type") == "game":
                    steam_rows.append({
                        "app_id": app_id,
                        "name": info.get("name"),
                        "type": info.get("type"),
                        "short_description": info.get("short_description"),
                        "release_date": info.get("release_date", {}).get("date"),
                        "required_age": info.get("required_age"),
                        "price_initial": (info.get("price_overview", {}).get("initial", 0) / 100)
                                         if info.get("price_overview") else None,
                        "price_final": (info.get("price_overview", {}).get("final", 0) / 100)
                                       if info.get("price_overview") else None,
                        "discount_percent": info.get("price_overview", {}).get("discount_percent")
                                            if info.get("price_overview") else None,
                        "platforms_windows": info.get("platforms", {}).get("windows"),
                        "platforms_mac": info.get("platforms", {}).get("mac"),
                        "platforms_linux": info.get("platforms", {}).get("linux"),
                        "genres": ", ".join([g["description"] for g in info.get("genres", [])]),
                        "categories": ", ".join([c["description"] for c in info.get("categories", [])]),
                        "metacritic_score": info.get("metacritic", {}).get("score"),
                        "recommendations_total": info.get("recommendations", {}).get("total"),
                        "developers": ", ".join(info.get("developers", [])),
                        "publishers": ", ".join(info.get("publishers", [])),
                        "screenshots_count": len(info.get("screenshots", [])),
                        "movies_count": len(info.get("movies", [])),
                        "dlc_count": len(info.get("dlc", [])),
                        "steam_url": f"https://store.steampowered.com/app/{app_id}/"
                    })
                    log(f"    ✓ Added game '{info.get('name')}'.")
                else:
                    log(f"    ✗ Skipped App ID {app_id} (type={info.get('type')}).")
                success = True
            else:
                log(f"    ✗ No valid data for App ID {app_id}.")
                success = True
        except requests.exceptions.RequestException as e:
            wait = INITIAL_WAIT * (2 ** retry)
            retry += 1
            log(f"    ! Retry {retry}/{MAX_RETRIES} for App ID {app_id} after {wait}s due to: {e}")
            time.sleep(wait)
        except Exception as e:
            log(f"    ! Non-retryable error for App ID {app_id}: {e}")
            break
    time.sleep(1.5)

steam_df = pd.DataFrame(steam_rows)
log(f"  → Completed Steam fetch: {len(steam_df)} records out of {len(app_ids)}.")

# ----------- Step 3: Fetch SteamSpy Data -----------
log("STEP 3: Fetching SteamSpy data for those games...")
steamspy_fields = [
    "name", "developer", "publisher", "score_rank",
    "positive", "negative", "average_playtime", "median_playtime",
    "owners", "owners_low", "owners_high", "owners_mid",
    "players_forever", "players_2weeks", "ccu"
]
for field in steamspy_fields:
    steam_df[f"steamspy_{field}"] = pd.NA

def parse_owners(owners_str):
    try:
        low_s, high_s = owners_str.split("-")
        low = int(low_s)
        high = int(high_s)
        mid = (low + high) // 2
        return low, high, mid
    except:
        return None, None, None

for idx, app_id in enumerate(steam_df["app_id"].astype(int), start=1):
    log(f"  [{idx}/{len(steam_df)}] Fetching SteamSpy info for App ID {app_id}...")
    try:
        url = f"https://steamspy.com/api.php?request=appdetails&appid={app_id}"
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        if data and int(data.get("appid", -1)) == app_id:
            steam_df.at[idx-1, "steamspy_name"] = data.get("name")
            steam_df.at[idx-1, "steamspy_developer"] = data.get("developer")
            steam_df.at[idx-1, "steamspy_publisher"] = data.get("publisher")
            steam_df.at[idx-1, "steamspy_score_rank"] = data.get("score_rank")
            steam_df.at[idx-1, "steamspy_positive"] = data.get("positive")
            steam_df.at[idx-1, "steamspy_negative"] = data.get("negative")
            steam_df.at[idx-1, "steamspy_average_playtime"] = data.get("average_playtime")
            steam_df.at[idx-1, "steamspy_median_playtime"] = data.get("median_playtime")
            owners = data.get("owners", "")
            steam_df.at[idx-1, "steamspy_owners"] = owners
            low, high, mid = parse_owners(owners or "")
            steam_df.at[idx-1, "steamspy_owners_low"] = low
            steam_df.at[idx-1, "steamspy_owners_high"] = high
            steam_df.at[idx-1, "steamspy_owners_mid"] = mid
            steam_df.at[idx-1, "steamspy_players_forever"] = data.get("players_forever")
            steam_df.at[idx-1, "steamspy_players_2weeks"] = data.get("players_2weeks")
            steam_df.at[idx-1, "steamspy_ccu"] = data.get("ccu")
            log("    ✓ SteamSpy data appended.")
        else:
            log("    ✗ No SteamSpy data returned.")
    except Exception as e:
        log(f"    ! ERROR fetching SteamSpy for App ID {app_id}: {e}")
    time.sleep(1.5)

# ----------- Step 4: One-hot Encoding Based on Master’s Columns -----------
log("STEP 4: One-hot encoding 'genres' and 'categories' based on master dataset structure...")

# Load master and ensure 'app_id' is int
master_df = pd.read_csv(MASTER_PATH)
master_df["app_id"] = master_df["app_id"].astype(int)
steam_df["app_id"] = steam_df["app_id"].astype(int)
log(f"  → Master has {len(master_df)} rows; this batch has {len(steam_df)} rows.")

# Detect existing one-hot columns in MASTER
GENRE_LABELS = [col.replace("genres_", "") for col in master_df.columns if col.startswith("genres_")]
CATEGORY_LABELS = [col.replace("categories_", "") for col in master_df.columns if col.startswith("categories_")]
log(f"  → Detected {len(GENRE_LABELS)} genre labels: {GENRE_LABELS[:5]}...")
log(f"  → Detected {len(CATEGORY_LABELS)} category labels: {CATEGORY_LABELS[:5]}...")

# One-hot encode STEAM batch according to those labels
def encode_batch(df, label_list, col, prefix):
    for lbl in label_list:
        cname = f"{prefix}_{lbl}"
        df[cname] = df[col].fillna("").apply(lambda x, lbl=lbl: int(lbl in [s.strip() for s in x.split(",")]))
    return df

steam_encoded = steam_df.copy()
log("  → One-hot encoding genres for batch...")
steam_encoded = encode_batch(steam_encoded, GENRE_LABELS, "genres", "genres")
log("  → One-hot encoding categories for batch...")
steam_encoded = encode_batch(steam_encoded, CATEGORY_LABELS, "categories", "categories")

# Drop raw 'genres' and 'categories' columns from batch
steam_encoded = steam_encoded.drop(columns=["genres", "categories"], errors="ignore")
log("  → Dropped raw 'genres' and 'categories' from batch after encoding.")

# Ensure MASTER has all columns from batch (fill missing with zeros)
log("  → Aligning batch columns to master...")
for col in master_df.columns:
    if col not in steam_encoded.columns:
        steam_encoded[col] = 0
        log(f"    → Added missing batch column '{col}' with zeros.")
# Ensure BATCH columns added to MASTER if missing (fill master with zeros)
for col in steam_encoded.columns:
    if col not in master_df.columns:
        master_df[col] = 0
        log(f"    → Added missing master column '{col}' with zeros.")
# Reorder steam_encoded to match master_df's column order
steam_encoded = steam_encoded[master_df.columns]
log("  → Column alignment complete.")

# ----------- Step 5: Merge and Log Changes -----------
log("STEP 5: Merging new batch into master and logging changes...")

existing_ids = set(master_df["app_id"].tolist())
new_mask = ~steam_encoded["app_id"].isin(existing_ids)
new_rows_df = steam_encoded[new_mask].copy()

before = len(master_df)
added = len(new_rows_df)
log(f"  → {added} new unique App IDs to add: {list(new_rows_df['app_id'])[:10]}...")

# Concatenate and dedupe
combined = pd.concat([master_df, new_rows_df], ignore_index=True)
combined = combined.drop_duplicates(subset="app_id").sort_values(by="app_id").reset_index(drop=True)
combined.to_csv(MASTER_PATH, index=False)
log(f"  → Master CSV updated: {before} → {len(combined)} rows (+{added}).")

# Append summary to log file
with open(LOG_PATH, "a") as logf:
    logf.write(f"\n=== {datetime.now()} ===\n")
    logf.write(f"Rows before merge: {before}\n")
    logf.write(f"Rows after merge:  {len(combined)}\n")
    logf.write(f"App IDs added this run: {list(new_rows_df['app_id'])}\n")
    logf.write("-" * 50 + "\n")

log(f"  → Merge changes recorded to '{LOG_PATH}'.")


[2025-06-03 17:23:13.291720] STEP 1: Sampling random App IDs from cleaned list...
[2025-06-03 17:23:13.563305]   → Loaded 249007 total App IDs.
[2025-06-03 17:23:13.569416]   → Sampled 1700 App IDs: [2297834, 3007300, 2222400, 2117110, 1254030, 1570840, 2475900, 597402, 2994470, 906130]...
[2025-06-03 17:23:13.569416] STEP 2: Fetching Steam storefront data for sampled App IDs...
[2025-06-03 17:23:13.570404]   [1/1700] Fetching Steam info for App ID 2297834...
[2025-06-03 17:23:13.802025]     ✗ Skipped App ID 2297834 (type=dlc).
[2025-06-03 17:23:15.302419]   [2/1700] Fetching Steam info for App ID 3007300...
[2025-06-03 17:23:15.565298]     ✗ Skipped App ID 3007300 (type=demo).
[2025-06-03 17:23:17.068669]   [3/1700] Fetching Steam info for App ID 2222400...
[2025-06-03 17:23:17.353718]     ✓ Added game 'Neighbors: Suburban Warfare - Playtest'.
[2025-06-03 17:23:18.856743]   [4/1700] Fetching Steam info for App ID 2117110...
[2025-06-03 17:23:19.090331]     ✓ Added game 'Trail of Toads

C:\Users\cmum9\AppData\Local\Temp\ipykernel_17724\586888704.py:201: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([master_df, new_rows_df], ignore_index=True)
